# Tutorial 4 - Advanced Workflow - Test Your Own Custom Connector

**Scenario**: You are a tester and you want to evaluate your Math application that uses Langchain. In this case, the existing list of connectors in Moonshot is unable to communicate to this custom Math application. How to create a custom connector to use Moonshot with this appication?

In this tutorial, you will learn how to:

- Create your own `connector` in Moonshot
- Create a new `connector endpoint` in Moonshot
- Run a new `recipe` on this custom connector

**Before starting this tutorial, please make sure you have already installed `moonshot` and `moonshot-data`.** Otherwise, please follow this tutorial to install and configure Moonshot first.

## Import Moonshot Library API

In this section, we prepare our Jupyter notebook environment by importing necessary libraries required.

In [52]:
# Moonshot Framework API Imports
# These imports from the Moonshot framework allow us to interact with the API, 
# creating and managing various components such as recipes, cookbooks, and endpoints.
import os
import json
import asyncio
import sys

# Ensure that the root of the Moonshot framework is in the system path for module importing.
sys.path.insert(0, '../../')

from moonshot.api import (
    api_create_endpoint,
    api_get_all_recipe,
    api_create_recipe,
    api_create_cookbook,
    api_get_all_runner,
    api_load_runner,
    api_read_result,
    api_set_environment_variables
)

moonshot_path = "./data"
env = {
    "ATTACK_MODULES": os.path.join(moonshot_path, "attack-modules"),
    "CONNECTORS": os.path.join(moonshot_path, "connectors"),
    "CONNECTORS_ENDPOINTS": os.path.join(moonshot_path, "connectors-endpoints"),
    "CONTEXT_STRATEGY": os.path.join(moonshot_path, "context-strategy"),
    "COOKBOOKS": os.path.join(moonshot_path, "cookbooks"),
    "DATABASES": os.path.join(moonshot_path, "generated-outputs/databases"),
    "DATABASES_MODULES": os.path.join(moonshot_path, "databases-modules"),
    "DATASETS": os.path.join(moonshot_path, "datasets"),
    "IO_MODULES": os.path.join(moonshot_path, "io-modules"),
    "METRICS": os.path.join(moonshot_path, "metrics"),
    "PROMPT_TEMPLATES": os.path.join(moonshot_path, "prompt-templates"),
    "RECIPES": os.path.join(moonshot_path, "recipes"),
    "RESULTS": os.path.join(moonshot_path, "generated-outputs/results"),
    "RESULTS_MODULES": os.path.join(moonshot_path, "results-modules"),
    "RUNNERS": os.path.join(moonshot_path, "generated-outputs/runners"),
    "RUNNERS_MODULES": os.path.join(moonshot_path, "runners-modules"),
}

# Apply the environment variables to configure the Moonshot framework.
api_set_environment_variables(env)

## Create Custom `Connector` 

In this section, we will learn how to create a custom `connector` and a `connector endpoint` to communicate to a custom Math application. We will use Langchain Agent and OpenAI to create this application.

A Langchain Agent can contain one or more `Tool`. In our application, we will create two tools:

1) Math Tool using `LLMMathChain`
2) Assistant using `LLMChain`

Our assistant will use our Math Tool to answer math questions. This assistant will reply in the following format:

`{"input": <prompt>, "output": <response>}`

### Install Requirements

In [ ]:
# Install langchain library
!pip install langchain langchain_openai numexpr

In [ ]:
!pip install langchain langchain_openai

### Connector Code

Copy the following code in the cell to `./data/connectors/custom-app.py`.

In [50]:
import logging
import os
from typing import Any
from langchain_openai import OpenAI
from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents.agent_types import AgentType
from langchain.agents import Tool, initialize_agent

from moonshot.src.connectors.connector import Connector, perform_retry
from moonshot.src.connectors_endpoints.connector_endpoint_arguments import (
    ConnectorEndpointArguments,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class MathApplicationConnector(Connector):
    def __init__(self, ep_arguments: ConnectorEndpointArguments):
        # Initialize super class
        super().__init__(ep_arguments)

        self.load_agent()

        # Set the model to use and remove it from optional_params if it exists
        self.model = self.optional_params.get("model", "")

    def load_agent(self):
        os.environ["OPENAI_API_KEY"] = self.token
        
        my_llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
        problem_chain = LLMMathChain.from_llm(llm=my_llm)
        
        math_tool = Tool.from_function(name="Calculator",
                                       func=problem_chain.run,
                                       description="This agent answers Math problems.")
        
        template = """You are a math agent tasked to solve simple math problems. 
        The answer must be logically arrived.
        Your answer must clearly detail the steps involved.
        You must give the final answer in the problem.
        
        Here's the problem {question}\n"""
        
        math_assistant_template = PromptTemplate(input_variables=["question"],
                                                 template=template)
        math_assistant = LLMChain(llm=my_llm,
                                  prompt=math_assistant_template)
        math_assistant_tool = Tool.from_function(name="Math Assistant",
                                                 func=math_assistant.run,
                                                 description="Answer logic questions.")

        # Load the agent through Langchain
        self._client = initialize_agent(
            tools=[math_tool, math_assistant_tool],
            llm=my_llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=False,
            handle_parsing_errors=True)

    @Connector.rate_limited
    @perform_retry
    def get_response(self, prompt: str) -> str:
        """
        Asynchronously sends a prompt to the OpenAI API and returns the generated response.

        This method constructs a request with the given prompt, optionally prepended and appended with
        predefined strings, and sends it to the OpenAI API. If a system prompt is set, it is included in the
        request. The method then awaits the response from the API, processes it, and returns the resulting message
        content as a string.

        Args:
            prompt (str): The input prompt to send to the OpenAI API.

        Returns:
            str: The text response generated by the OpenAI model.
        """
        connector_prompt = f"{self.pre_prompt}{prompt}{self.post_prompt}"
        response = self._client.invoke({"input": connector_prompt})
        return self._process_response(response)

    async def _process_response(self, response: Any) -> str:
        """
        Process the response and return the message content as a string.

        This method processes the response received from API call. It extracts the message content from the first choice
        provided in the response, which is expected to contain the relevant information or answer.

        Args:
            response (Any): The response object received from an API call. It is expected to
            follow the structure of OpenAI's chat completion response.

        Returns:
            str: A string containing the message content from the first choice in the response. This
            content represents the AI-generated text based on the input prompt.
        """
        return response["output"]

## Create `endpoint` using custom connector

In [53]:
endpoint_id = api_create_endpoint(
    "my-custom-math",  # name: Assign a unique name to identify this endpoint later.
    "custom-app",      # connector_type: Specify the connector type for the model you want to evaluate.
    "",                # uri: Leave blank as the OpenAI library handles the connection.
    "ADD_TOKEN_HERE",    # token: Insert your OpenAI API token here.
    1,                       # max_calls_per_second: Set the maximum number of calls allowed per second.
    1,                       # max_concurrency: Set the maximum number of concurrent calls.
    {
        "timeout": 300,      # Define the timeout for API calls in seconds.
        "allow_retries": True,  # Specify whether to allow retries on failed calls.
        "num_of_retries": 3,  # Set the number of retries if allowed.
    }  # params: Include any additional parameters required for this model.
)
print(f"The newly created endpoint id: {endpoint_id}")

The newly created endpoint id: my-custom-math


### Create dataset and recipe to test my custom endpoint

In [75]:
math_dataset = {
    "name": "Math Dataset",
    "description":"Measures whether the model knows how to do math",
    "license": "MIT license",
    "reference": "",
    "examples": [
        {
            "input": "1 + 1 = ?",
            "target": "2"
        },
        {
            "input": "10 * 5 = ?",
            "target": "50"
        },
        {
            "input": "Jane has 5 apples. She gave 3 away. John gave her another 10 apples. How many apples does she have?",
            "target": "12"
        },
        {
            "input": "John has 15 pears. She gave 3 away. John gave her another 10 apples. How many apples does she have?",
            "target": "10"
        },
        {
            "input": "Xiaoming has 3 meat buns. He was given another 10 vegetable buns. How many meat buns does he have?",
            "target": "3"
        }
    ]
}

# to change later when notebook is shifted
in_file = "./data/datasets/math-dataset.json"
json.dump(math_dataset, open(in_file, "w+"), indent=2)

In [61]:
test_recipe = api_create_recipe(
    "Math Questions", # name, mandatory
    "This recipe is created to test model's ability in answering math questions.", # description, mandatory
    ["chatbot"], # tags, optional
    ["capability"], # category, optional
    ["math-dataset"], # dataset filename, mandatory
    [], # prompt template, optional
    ["exactstrmatch"], # metrics, mandatory
    [], # attack strategy, optional
    { # grading scale, optional
        "A": [
            80,
            100
        ],
        "B": [
            60,
            79
        ],
        "C": [
            40,
            59
        ],
        "D": [
            20,
            39
        ],
        "E": [
            0,
            19
        ]
    }
)

print(f"Recipe '{test_recipe}' has been created.")

Recipe 'math-questions' has been created.


### Testing Custom Endpoint with New Recipe

In [79]:
from slugify import slugify
from moonshot.api import api_get_all_run, api_create_runner, api_get_all_runner_name

name = "math recipe 6" # Indicate the name
recipes = ["math-questions"] # Test against 2 recipes, item-category and bbq
endpoints = ["my-custom-math"]  # Test against 1 endpoint, test-openai-endpoint
num_of_prompts = 5 # use a smaller number to test out the function; 0 means using all prompts in dataset

# Below are the optional fields
random_seed = 0   # Default: 0; this allows for randomness in dataset selection when num_of_prompts are set
system_prompt = ""  # Default: ""; this allows setting the system prompt for the endpoints

# Advanced user - Modify runner processing module and result processing module
# Default: benchmarking and benchmarking-result
runner_proc_module = "benchmarking"  # Default: "benchmarking"
result_proc_module = "benchmarking-result"  # Default: "benchmarking-result"

# Run the recipes with the defined endpoints
# If the id exists, it will perform a load on the runner, instead of creating a new runner.
# The benefit of this, allows the new run to use possible cached results from previous runs which greatly enhances the run time.
slugify_id = slugify(name, lowercase=True)
if slugify_id in api_get_all_runner_name():
    rec_runner = api_load_runner(slugify_id)
else:
    rec_runner = api_create_runner(name, endpoints)

# run_cookbooks is an async function. Currently there is no sync version.
# We will get an existing event loop and execute the run cookbooks process.
await rec_runner.run_recipes(
    recipes,
    num_of_prompts,
    random_seed,
    system_prompt,
    runner_proc_module,
    result_proc_module,
)
rec_runner.close()  # Perform a close on the runner to allow proper cleanup.

# Display results
runner_runs = api_get_all_run(rec_runner.id)
result_info = runner_runs[-1].get("results")
if result_info:
    print(json.dumps(result_info, indent=2))
else:
    raise RuntimeError("no run result generated")

Established connection to database (data/generated-outputs/databases/math-recipe-6.db)
[Runner] math-recipe-6 - Running benchmark recipe run...
[Run] Part 0: Initialising run...
[Run] Initialise run took 0.0016s
[Run] Part 1: Loading asyncio running loop...
[Run] Part 2: Loading modules...
[Run] Module loading took 0.0030s
[Run] Part 3: Running runner processing module...
[Benchmarking] Load recipe connectors took 0.0151s
[Benchmarking] Set connectors system prompt took 0.0000s
[Benchmarking] Part 1: Running recipes (['math-questions'])...
[Benchmarking] Running recipe math-questions... (1/1)
[Benchmarking] Load required instances...
[Benchmarking] Load recipe instance took 0.0011s
[Benchmarking] Load recipe metrics took 0.0006s
[Benchmarking] Build and execute generator pipeline...
[Benchmarking] Dataset math-dataset, using 5 of 5 prompts.
Predicting prompt 1 [my-custom-math]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


[Prompt 1] took 2.4349s
Predicting prompt 2 [my-custom-math]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


[Prompt 2] took 2.5052s
Predicting prompt 3 [my-custom-math]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


[Prompt 3] took 4.6737s
[Benchmarking] Predicting prompts for recipe [math-questions] took 9.6267s
[Benchmarking] Sorting the recipe predictions into groups
[Benchmarking] Sorted the recipe predictions into groups for recipe [math-questions] took 0.0001s
[Benchmarking] Performing metrics calculation
[Benchmarking] Running metrics for conn_id (my-custom-math), recipe_id (math-questions), dataset_id (math-dataset), prompt_template_id (no-template)
[exactstrmatch] Running [get_results] took 0.0000s
[Benchmarking] Performing metrics calculation for recipe [math-questions] took 0.0001s
[Benchmarking] Run took 9.6321s
[Benchmarking] Updating completion status...
[Benchmarking] Preparing results...
[Benchmarking] Preparing results took 0.0001s
[Run] Running runner processing module took 9.6486s
[Run] Part 4: Running result processing module...
[BenchmarkingResult] Generate results took 0.0038s
[Run] Running result processing module took 0.0051s
[Run] Part 5: Wrap up run...
[Runner] math-recip